In [40]:
import pandas as pd 
import numpy as np
import spacy 

In [41]:
df_T=pd.read_csv("True.csv")

In [42]:
df_F=pd.read_csv("Fake.csv")

In [43]:
df_T.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [44]:
df_T["label"]="Real"

In [46]:
df_T.drop(["date"],axis=1,inplace=True)
df_T.drop(["title"],axis=1,inplace=True)
df_T.drop(["subject"],axis=1,inplace=True)


In [47]:

df_T.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,Real
1,WASHINGTON (Reuters) - Transgender people will...,Real
2,WASHINGTON (Reuters) - The special counsel inv...,Real
3,WASHINGTON (Reuters) - Trump campaign adviser ...,Real
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,Real


In [48]:
df_F['label']='Fake'

In [49]:
df_F.drop(["date"],axis=1,inplace=True)
df_F.drop(["subject"],axis=1,inplace=True)
df_F.drop(["title"],axis=1,inplace=True)


In [50]:
df_F.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,Fake
1,House Intelligence Committee Chairman Devin Nu...,Fake
2,"On Friday, it was revealed that former Milwauk...",Fake
3,"On Christmas day, Donald Trump announced that ...",Fake
4,Pope Francis used his annual Christmas Day mes...,Fake


In [52]:
df=df_T.merge(df_F, how='outer')

In [53]:
df.shape

(44898, 2)

In [55]:
df.columns


Index(['text', 'label'], dtype='object')

In [56]:
df.describe

<bound method NDFrame.describe of                                                     text label
0      WASHINGTON (Reuters) - The head of a conservat...  Real
1      WASHINGTON (Reuters) - Transgender people will...  Real
2      WASHINGTON (Reuters) - The special counsel inv...  Real
3      WASHINGTON (Reuters) - Trump campaign adviser ...  Real
4      SEATTLE/WASHINGTON (Reuters) - President Donal...  Real
...                                                  ...   ...
44893  21st Century Wire says All the world s a stage...  Fake
44894    Randy Johnson 21st Century WireThe majority ...  Fake
44895  Tune in to the Alternate Current Radio Network...  Fake
44896  21st Century Wire says A new front has just op...  Fake
44897  21st Century Wire says It s not that far away....  Fake

[44898 rows x 2 columns]>

In [57]:
df.shape

(44898, 2)

In [58]:
df.label.value_counts()

Fake    23481
Real    21417
Name: label, dtype: int64

In [59]:
df['label_num']=df['label'].map({'Real':1,'Fake':0})

In [60]:
df.head()

,text,label,label_num
0,WASHINGTON (Reuters) - The head of a conservat...,Real,1
1,WASHINGTON (Reuters) - Transgender people will...,Real,1
2,WASHINGTON (Reuters) - The special counsel inv...,Real,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,Real,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,Real,1


In [61]:
import spacy

In [62]:
nlp=spacy.load('en_core_web_lg')

In [63]:
df['vector']=df['text'].apply(lambda x:nlp(x).vector)

In [64]:
df.shape

(44898, 4)

In [67]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    df['vector'],
    df['label_num'], 
    test_size=0.2,
    random_state=2223)

In [68]:
 X_train

33843    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
21401    [-1.6966594, 0.41185793, -1.861141, 0.96591526...
14534    [-2.191039, 0.3886821, -1.4236075, 0.6471172, ...
8996     [-1.9861172, 0.75599295, -1.8335453, 0.9438286...
18402    [-1.6998668, 0.8086158, -2.1879313, -0.0142487...
                               ...                        
20142    [-2.2564137, 0.03532076, -1.3432753, 0.2537124...
24120    [-1.980588, -0.017904833, -1.9464817, 0.290707...
35242    [-0.97709346, 2.516167, -1.4206401, 0.5265695,...
21823    [-2.4994774, 0.040455483, -2.3414872, 0.390758...
15196    [-1.786645, 0.80069953, -1.5167662, 1.3180462,...
Name: vector, Length: 35918, dtype: object

In [69]:
X_train_2d=np.stack(X_train)
X_test_2d=np.stack(X_test)

In [74]:

X_train_2d.shape

(35918, 300)

In [85]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


In [86]:
clf=MultinomialNB()

In [89]:
scaler=MinMaxScaler()
scaled_train=scaler.fit_transform(X_train_2d)
scaled_test=scaler.fit_transform(X_test_2d)

clf.fit(scaled_train,y_train)

MultinomialNB()

In [79]:
y_train.shape

(35918,)

In [90]:
y_pred=clf.predict(scaled_test)

In [96]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.65      0.78      4689
           1       0.72      0.97      0.83      4291

    accuracy                           0.81      8980
   macro avg       0.84      0.81      0.80      8980
weighted avg       0.85      0.81      0.80      8980



In [95]:
from sklearn.metrics import classification_report

In [97]:
from sklearn.neighbors import KNeighborsClassifier

In [127]:
clf=KNeighborsClassifier(n_neighbors=3, metric='euclidean')

In [128]:
clf.fit(scaled_train,y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=3)

In [129]:
y_pred=clf.predict(X_test_2d)

In [130]:
scaled_test.shape

(8980, 300)

In [131]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84      4689
           1       0.87      0.75      0.80      4291

    accuracy                           0.83      8980
   macro avg       0.83      0.82      0.82      8980
weighted avg       0.83      0.83      0.82      8980

